-What about electrons? (I don't see any - 2 phys.me)

-1e11 and mDM?

-Compute T(E,z_in, z_dep) by doing a delta function injection

# Full Transfer Functions

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [95]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from astropy.io import fits

import numpy as np
from numpy.linalg import matrix_power
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_spectrum import nonrel_spec
from darkhistory.electrons.ics.ics_spectrum import rel_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

from darkhistory.electrons import positronium as pos

from darkhistory.low_energy.lowE_deposition import compute_fs

# np.set_printoptions(threshold=np.nan)

In [3]:
soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
xe_std  = interp1d(soln[0,:], soln[2,:])
T_m_std = interp1d(soln[0,:], soln[1,:])

## Photon Transfer Functions

In [4]:
user = 'hongwan'

highengphot_tflist_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_photspec_60eV_complete_coarse.raw", "rb"))
lowengphot_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec_60eV_complete_coarse.raw", "rb"))
lowengelec_tflist_arr  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec_60eV_complete_coarse.raw", "rb"))
CMB_engloss_arr = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/CMB_engloss_60eV_complete_coarse.raw", "rb"))

In [5]:
photeng = highengphot_tflist_arr[0].eng
eleceng = lowengelec_tflist_arr[0].eng

#Split photeng into high and low energy. 
photeng_high = photeng[photeng > 60]
photeng_low  = photeng[photeng <= 60]

# Split eleceng into high and low energy. 
eleceng_high = eleceng[eleceng > 3000]
eleceng_low  = eleceng[eleceng <= 3000]


for highengphot_tflist in highengphot_tflist_arr:
    
#     highengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)

    for tf in highengphot_tflist:
        # tf.rebin(photeng_high)
#         tf.rebin(photeng)
        
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size, 0), (0, 0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    highengphot_tflist._eng = photeng
    highengphot_tflist._in_eng = photeng
    highengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in highengphot_tflist._tflist])
    )
    
# lowengphot_tflist.in_eng set to photeng_high
for lowengphot_tflist in lowengphot_tflist_arr:
    
#     lowengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengphot_tflist:
#         tf.rebin(photeng)
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        # Photons in the low energy bins should be immediately deposited.
        tf._grid_vals[0:photeng_low.size, 0:photeng_low.size] = np.identity(photeng_low.size)
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    lowengphot_tflist._eng = photeng
    lowengphot_tflist._in_eng = photeng
    lowengphot_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengphot_tflist._tflist])
    )
    
# lowengelec_tflist.in_eng set to photeng_high 
for lowengelec_tflist in lowengelec_tflist_arr:
    
#     lowengelec_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengelec_tflist:
#         tf.rebin(eleceng)
        # Pad with zeros so that it becomes photeng x eleceng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = eleceng
        tf._rs = tf.rs[0]*np.ones_like(photeng)

    lowengelec_tflist._eng = eleceng
    lowengelec_tflist._in_eng = photeng
    lowengelec_tflist._grid_vals = np.atleast_3d(
        np.stack([tf.grid_vals for tf in lowengelec_tflist._tflist])
    )
    
    for engloss in CMB_engloss_arr:
        engloss = np.pad(engloss, ((0,0),(photeng_low.size, 0)), 'constant')


In [6]:
xes = 0.5 + 0.5*np.tanh([-5., -4.1, -3.2, -2.3, -1.4, -0.5, 0.4, 1.3, 2.2, 3.1, 4])

highengphot_tf_interp = tflist.TransferFuncInterp(xes, highengphot_tflist_arr)
lowengphot_tf_interp = tflist.TransferFuncInterp(xes, lowengphot_tflist_arr)
lowengelec_tf_interp = tflist.TransferFuncInterp(xes, lowengelec_tflist_arr)


## Inverse Compton Scattering

Set up all of the inverse Compton scattering transfer functions (needs to be hidden away at some point). 

In [7]:
# %%prun

# Emax = 1e20
# Emin = 1e-8
# nEe = 5000
# nEp  = 5000

# dlnEp = np.log(Emax/Emin)/nEp
# lowengEp_rel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)        

# dlnEe = np.log(Emax/Emin)/nEe
# lowengEe_rel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

# Emax = 1e10
# Emin = 1e-8
# nEe = 5000
# nEp  = 5000

# dlnEp = np.log(Emax/Emin)/nEp
# lowengEp_nonrel = Emin*np.exp((np.arange(nEp)+0.5)*dlnEp)  

# dlnEe = np.log(Emax/Emin)/nEe
# lowengEe_nonrel = Emin*np.exp((np.arange(nEe)+0.5)*dlnEe)

# print('********* Thomson regime scattered photon spectrum *********')
# ics_thomson_ref_tf = nonrel_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(400))
# print('********* Relativistic regime scattered photon spectrum *********')
# ics_rel_ref_tf = rel_spec(lowengEe_rel, lowengEp_rel, phys.TCMB(400), inf_upp_bound=True)
# print('********* Thomson regime energy loss spectrum *********')
# engloss_ref_tf = engloss_spec(lowengEe_nonrel, lowengEp_nonrel, phys.TCMB(400), nonrel=True)


In [96]:
def scattered_elec_spec(
    eleckineng, delta, T, 
    as_pairs=False, nonrel=False, nonrel_tf=None, rel_tf=None,
    spec_type='new'
):
    """ Energy loss ICS spectrum. 

    Switches between `engloss_spec_series` and `engloss_spec_diff`. 

    Parameters
    ----------
    eleckineng : ndarray
        Incoming electron kinetic energy. 
    delta : ndarray
        Energy gained by photon after upscattering (only positive values). 
    T : float
        CMB temperature.
    as_pairs : bool, optional
        If true, treats eleckineng and photeng as a paired list: produces eleckineng.size == photeng.size values. Otherwise, gets the spectrum at each photeng for each eleckineng, returning an array of length eleckineng.size*photeng.size. 
    nonrel : bool, optional
        If true, only returns the nonrelativistic energy loss spectrum, and never switches to the relativistic case. 
    nonrel_tf : TransFuncAtRedshift, optional
        Reference nonrelativistic energy loss ICS spectrum. If specified, calculation is done by interpolating over the transfer function. 
    rel_tf : TransFuncAtRedshift, optional
        Reference relativistic energy loss ICS spectrum. If specified, calculation is done by interpolating over the transfer function. 
    spec_type : {'old', 'new'}
        The ICS secondary photon spectrum to integrate over. 

    Returns
    -------
    TransFuncAtRedshift or ndarray
        dN/(dt d Delta) of the outgoing photons (dt = 1 s). If as_pairs == False, returns a TransFuncAtRedshift, with abscissa given by (eleckineng, delta). Otherwise, returns an ndarray, with abscissa given by each pair of (eleckineng, delta). 
    """

    gamma = eleckineng/phys.me + 1
    eleceng = eleckineng + phys.me
    beta = np.sqrt(eleckineng/phys.me*(gamma+1)/gamma**2)
    eta = delta/T

    # where to switch between nonrelativistic and relativistic treatments.
    if nonrel:
        rel_bound = np.inf 
    else:
        rel_bound = 20

    # 2D masks have dimensions (eleceng, delta).

    if as_pairs:
        if eleckineng.size != delta.size:
            raise TypeError('delta and electron energy arrays must have the same length for pairwise computation.')
        gamma_mask = gamma
        beta_mask = beta
        eleckineng_mask = eleckineng 
        eleceng_mask = eleceng
        delta_mask = delta
        spec = np.zeros_like(gamma)
    else:
        gamma_mask = np.outer(gamma, np.ones_like(eta))
        beta_mask = np.outer(beta, np.ones_like(eta))
        eleckineng_mask = np.outer(eleckineng, np.ones_like(eta))
        eleceng_mask = np.outer(eleceng, np.ones_like(eta))
        delta_mask = np.outer(np.ones_like(eleckineng), delta)
        spec = np.zeros(
            (eleckineng.size, delta.size), dtype='float128'
        )

    # beta_small = beta_mask < 0.05
    if spec_type == 'old':
        beta_small = beta_mask < 0.05
    if spec_type == 'new':
        beta_small = beta_mask < 0.1
    rel = gamma_mask > rel_bound

    y = T/phys.TCMB(400)

    if not nonrel:
        if rel_tf != None:
            if as_pairs:
                raise TypeError('When reading from file, the keyword as_pairs is not supported.')
            # If the electron energy at which interpolation is to be taken is outside rel_tf, then an error should be returned, since the file has not gone up to high enough energies.
            #rel_tf = rel_tf.at_in_eng(y*eleceng[gamma > rel_bound])
            # If the photon energy at which interpolation is to be taken is outside rel_tf, then for large photon energies, we set it to zero, since the spectrum should already be zero long before. If it is below, nan is returned, and the results should not be used.
            # rel_tf = rel_tf.at_eng(
            #     y*delta, 
            #     bounds_error = False,
            #     fill_value = (np.nan, 0)
            # )

            # OLD METHOD: call at_val

            # rel_tf = rel_tf.at_val(
            #     y*eleceng[gamma > rel_bound], y*delta, 
            #     bounds_error=False, fill_value = 1e-200
            # )
            # spec[rel] = y**4*rel_tf.grid_vals.flatten()

            # NEW METHOD: call interpolator
            rel_tf_interp = rel_tf.interp_func(
                np.log(y*delta),
                np.log(y*eleceng[gamma > rel_bound])
            )
            spec[rel] = y**4*rel_tf_interp.flatten()

        else:

            print('Computing relativistic energy loss spectrum...')

            spec[rel] = ics_spectrum.rel_spec(
                eleceng_mask[rel],
                delta_mask[rel],
                T, inf_upp_bound=True, as_pairs=True 
            )

            print('Relativistic energy loss spectrum complete!')

    if nonrel_tf != None:
        # nonrel_tf = nonrel_tf.at_in_eng(eleceng[gamma <= rel_bound] - phys.me)
        # nonrel_tf = nonrel_tf.at_eng(
        #     delta/y,
        #     bounds_error = False,
        #     fill_value = (np.nan, 0)
        # )

        # OLD METHOD: call at_val
        # nonrel_tf = nonrel_tf.at_val(
        #     eleckineng[gamma <= rel_bound], delta/y,
        #     bounds_error = False, fill_value = 1e-200
        # )
        # spec[~rel] = y**2*nonrel_tf.grid_vals.flatten()

        # NEW METHOD: call interpolator
        nonrel_tf_interp = nonrel_tf.interp_func(
            np.log(delta/y),
            np.log(eleckineng[gamma <= rel_bound])
        )
        spec[~rel] = y**2*nonrel_tf_interp.flatten()

    else:
        print('Computing nonrelativistic energy loss spectrum...')
        # beta_small obviously doesn't intersect with rel. 
        spec[beta_small] = engloss_spec_diff(
            eleckineng_mask[beta_small],
            delta_mask[beta_small],
            T, as_pairs=True, spec_type=spec_type
        )

        spec[~beta_small & ~rel] = engloss_spec_series(
            eleckineng_mask[~beta_small & ~rel],
            delta_mask[~beta_small & ~rel],
            T, as_pairs=True, spec_type=spec_type
        )
        print('Nonrelativistic energy loss spectrum computed!')

    # Zero out spec values that are too small (clearly no scatters within the age of the universe), and numerical errors. Non-zero to take log interpolation later. 
    spec[spec < 1e-100] = 0.
    
    if as_pairs:
        return spec 
    else:

        rs = T/phys.TCMB(1)
        dlnz = 1/(phys.dtdz(rs)*rs)

        spec_arr = [
            Spectrum(delta, sp, rs=rs, in_eng=in_eng) 
            for sp, in_eng in zip(spec, eleckineng)
        ]

        return TransFuncAtRedshift(
            spec_arr, dlnz=dlnz, 
            in_eng = eleckineng, eng = delta,
            with_interp_func=True
        )

[8.84136730e+06 8.76838141e+06 8.69599803e+06 ... 0.00000000e+00
 1.16415322e-10 1.16415322e-10]
float128
[8.91496070e+06 1.77563280e+07 2.65247094e+07 ... 1.07994093e+09
 1.07994093e+09 1.07994093e+09]


TypeError: abscissa must be ordered in increasing energy.

In [97]:
np.float128(1e20) - np.float128(1e-8)

1e+20

In [79]:
# pickle.dump(ics_thomson_ref_tf, open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_thomson_ref_tf.p", "wb"))
# pickle.dump(ics_rel_ref_tf,     open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_rel_ref_tf.p", "wb"))
# pickle.dump(engloss_ref_tf,     open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/engloss_ref_tf.p", "wb"))

In [9]:
ics_thomson_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_thomson_ref_tf.p", "rb"))
ics_rel_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ics_rel_ref_tf.p", "rb"))
engloss_ref_tf_2 = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/engloss_ref_tf.p", "rb"))

In [22]:
%%prun
start_rs = 3000
rs = start_rs
while rs > 2500:
    (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
        ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
        eleceng, photeng, rs, fast=True
    )
    rs = np.exp(np.log(rs) - 0.002)
    print(rs)

2994.005996001995
2988.023968031968
2982.053892161797
2976.095744511174
2970.149501247497
2964.215138585785
2958.29263278858
2952.38196016585
2946.4830970748976
2940.5960199202623
2934.720705153627
2928.8571292737256
2923.0052688262463
2917.1651004037394
2911.336600645524
2905.5197462375927
2899.714513912521
2893.9208804493705
2888.1388226736008
2882.3683174569724
2876.609341717457
2870.8618724191438
2865.1258865721484
2859.4013612325193
2853.6882735021477
2847.9866005286754
2842.2963195054017
2836.6174076711964
2830.9498423104037
2825.293600752755
2819.648660373276
2814.0149985921985
2808.392592874867
2802.781420731652
2797.1814597178563
2791.5926874336296
2786.0150815238744
2780.4486196781604
2774.893279630632
2769.349039159922
2763.8158760890615
2758.29376828539
2752.7826936604692
2747.282630169994
2741.7935558137024
2736.3154486352896
2730.8482867223197
2725.3920482061376
2719.9467112617826
2714.512254107899
2709.0886550066516
2703.675892263636
2698.2739442277943
2692.8827892913273

## Evolution

In [46]:
# mDM = 605662.1322589782*2
# mDM = 10**9.104*2
# mDM = 1.27164e9*2
mDM = 10**7*2
# mDM = 10**6.5*2

phot_spec_init = spectools.rebin_N_arr(np.array([2]), 
                                      np.array([mDM/2]), 
                                      photeng)
phot_spec_init.switch_spec_type()

elec_spec_init = spectools.rebin_N_arr(np.array([2]), 
    np.array([mDM/2 - phys.me]),
    eleceng)

elec_spec_init.switch_spec_type()

In [47]:
print(phot_spec_init.toteng(bound_type = 'eng', bound_arr = np.array([1,1e13]))/mDM)

[1.]


In [48]:
%autoreload
phot_spec_init.rs = 3000
elec_spec_init.rs = 3000
end_rs   = 4
#mDM = 1e8
sigmav = 1e-26
tau = 1e25

def rate_func_swave_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)/(2*mDM)

def rate_func_swave_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav)

def rate_func_decay_N(rs):
    # This is dN/dV dt
    return phys.inj_rate('decay', rs, tau=tau)/mDM

def rate_func_decay_eng(rs):
    # This is dE/dV dt
    return phys.inj_rate('decay', rs, tau=tau)


def rate_func_swave_N_2(rs):
    # This is dN/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)/(2*mDM)

def rate_func_swave_eng_2(rs):
    # This is dE/dV dt
    return phys.inj_rate('swave', rs, mDM=mDM, sigmav=sigmav*1e-30)


def evolve(
    in_spec_elec, in_spec_phot, 
    rate_func_N, rate_func_eng, xe_init, T_m_init,
    highengphot_tf_interp, lowengphot_tf_interp, 
    lowengelec_tf_interp, 
    end_rs, coarsen_factor=1, std_soln=False
):
    """
    Main function that computes the temperature and ionization history. 
    
    Parameters
    ----------
    in_spec_elec : Spectrum
        Spectrum per annihilation/decay into electrons. 
    in_spec_phot : Spectrum
        Spectrum per annihilation/decay into photons.
    rate_func_N : function
        Function describing the rate of annihilation/decay, dN/(dV dt)
    rate_func_eng : function
        Function describing the rate of annihilation/decay, dE/(dV dt)
    xe_init : float
        xe at the initial redshift. 
    T_m_init : float
        Matter temperature at the initial redshift. 
    highengphot_tf_interp : TransferFuncInterp
        Interpolation table for high energy photons. 
    lowengphot_tf_interp : TransferFuncInterp
        Interpolation table for low energy photons. 
    lowengelec_tf_interp : TransferFuncInterp
        Interpolation table for low energy electrons. 
    end_rs : float
        Final redshift to evolve to. 
    coarsen_factor : int
        Coarsening to apply to the transfer function matrix. 
    std_soln : bool
        If true, uses the standard TLA solution for f(z). 
    """
    
    # Initialize the next spectrum as None.
    next_highengphot_spec = None
    next_lowengphot_spec  = None
    next_lowengelec_spec  = None
    
    if (
        highengphot_tf_interp.dlnz    != lowengphot_tf_interp.dlnz
        or highengphot_tf_interp.dlnz != lowengelec_tf_interp.dlnz
        or lowengphot_tf_interp.dlnz  != lowengelec_tf_interp.dlnz
    ):
        raise TypeError('TransferFuncInterp objects must all have the same dlnz.')
    
    if in_spec_elec.rs != in_spec_phot.rs:
        raise TypeError('Input spectra must have the same rs.')
    
    # redshift/timestep related quantities. 
    dlnz = highengphot_tf_interp.dlnz
    prev_rs = None
    rs = in_spec_phot.rs
    dt = dlnz*coarsen_factor/phys.hubble(rs)
    
    # Function that changes the normalization from per annihilation to per baryon. 
    def norm_fac(rs): 
        return rate_func_N(rs) * dlnz * coarsen_factor /phys.hubble(rs) / (phys.nB * rs**3)
    
    # ICS for in_spec_elec goes here.
    
    (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
        ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
        eleceng, photeng, rs, fast=True
    )
    
    ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec) * norm_fac(rs)
    ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec) * norm_fac(rs)
    positronium_phot_spec = pos.weighted_photon_spec(photeng)*in_spec_elec.totN()/2*norm_fac(rs)
    if positronium_phot_spec.spec_type != 'N':
        positronium_phot_spec.switch_spec_type()
        
    print('input electron energy: ', (phys.me*2 + in_spec_elec.toteng())*norm_fac(rs))
    print('ICS secondary photon spectrum energy: ', ics_phot_spec.toteng())
    print('Upscattered photon energy: ', np.dot(in_spec_elec.N, continuum_loss)*norm_fac(rs))
    print('ICS low energy electron spectrum energy: ', ics_lowengelec_spec.toteng() + phys.me*norm_fac(rs))
    print('Positronium spectrum energy: ', positronium_phot_spec.toteng())
    
    init_inj_spec = in_spec_phot * norm_fac(rs) + ics_phot_spec + positronium_phot_spec
    
    # Initialize the Spectra object that will contain all the 
    # output spectra during the evolution.
    out_highengphot_specs = Spectra([init_inj_spec], spec_type=init_inj_spec.spec_type)
    out_lowengphot_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    out_lowengelec_specs  = Spectra([], spec_type=init_inj_spec.spec_type)
    
    # Initialize the xe and T array that will store the solutions.
    xe_arr  = np.array([xe_init])
    T_m_arr = np.array([T_m_init])
    
    # Load the standard TLA solution if necessary. 
#     if std_soln:
    if True:
        soln = pickle.load(open("../darkhistory/history/std_soln.p", "rb"))
        xe_std  = interp1d(soln[0,:], soln[2,:])
        T_m_std = interp1d(soln[0,:], soln[1,:])

    # Define these methods for speed.
    append_highengphot_spec = out_highengphot_specs.append
    append_lowengphot_spec  = out_lowengphot_specs.append
    append_lowengelec_spec  = out_lowengelec_specs.append
    
    # For the tqdm bar
    pbar = tqdm(total = (np.log(rs) - np.log(end_rs))/dlnz/coarsen_factor)
    
    # Loop while we are still at a redshift above end_rs.
    while rs > end_rs:
        
        # Update tqdm bar.
        pbar.update(1)
        
        # If prev_rs exists, calculate xe and T_m. 
        if prev_rs is not None:
#             print('here at least?')
            
            # f_continuum, f_lyman, f_ionH, f_ionHe, f_heat
            # f_raw takes in dE/(dV dt)
            if std_soln:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )
#                 print("\n SHOULD BE ONE: ", (next_lowengelec_spec.toteng() + next_lowengphot_spec.toteng())/(
#                 np.sum(f_raw) * rate_func_eng(rs) * dt / phys.nB / rs**3), '\n')   
            else:
                f_raw = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
                
            std_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_std(rs), 0, 0]), rate_func_eng(rs), dt
                )
            
            backreaction_f = compute_fs(
                    next_lowengelec_spec, next_lowengphot_spec,
                    np.array([1-xe_arr[-1], 0, 0]), rate_func_eng(rs), dt
                )
            
            f_raw = np.float64(f_raw)
            
            init_cond = np.array([T_m_arr[-1], xe_arr[-1], 0, 0])

            new_vals = tla.get_history(
                init_cond, f_raw[2], f_raw[1], f_raw[4], 
                rate_func_eng, np.array([prev_rs, rs]),
                reion_switch = False
            )

            T_m_arr = np.append(T_m_arr, new_vals[-1,0])
            xe_arr  = np.append(xe_arr,  new_vals[-1,1])
            
        print('T_m at '+str(rs)+': '+ str(T_m_arr[-1]))
        print('x_e at '+str(rs)+': '+ str(xe_arr[-1]))
        print('Standard x_e at '+str(rs)+': '+str(xe_std(rs)))
        print('Standard T_m at '+str(rs)+': '+str(T_m_std(rs)))
        if prev_rs is not None:
            print('Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', backreaction_f)
            print('Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat: ', std_f)
            
        if std_soln:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_std(rs))
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_std(rs))
        else:
            highengphot_tf = highengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengphot_tf  = lowengphot_tf_interp.get_tf(rs, xe_arr[-1])
            lowengelec_tf  = lowengelec_tf_interp.get_tf(rs, xe_arr[-1])
        
        if coarsen_factor > 1:
            prop_tf = np.zeros_like(highengphot_tf._grid_vals)
            for i in np.arange(coarsen_factor):
                prop_tf += matrix_power(highengphot_tf._grid_vals, i)
            lowengphot_tf._grid_vals = np.matmul(prop_tf, lowengphot_tf._grid_vals)
            lowengelec_tf._grid_vals = np.matmul(prop_tf, lowengelec_tf._grid_vals)
            highengphot_tf._grid_vals = matrix_power(
                highengphot_tf._grid_vals, coarsen_factor
            )
            
        next_highengphot_spec = highengphot_tf.sum_specs(out_highengphot_specs[-1])
        next_lowengphot_spec  = lowengphot_tf.sum_specs(out_highengphot_specs[-1])
        next_lowengelec_spec  = lowengelec_tf.sum_specs(out_highengphot_specs[-1]) + ics_lowengelec_spec
        
#         print('\n SHOULD ALSO BE ONE: ', 
#               out_highengphot_specs[-1].toteng(bound_type = 'eng', bound_arr = np.array([100,1e13]))/(
#         next_highengphot_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13])) + 
#             next_lowengphot_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13])) + 
#             next_lowengelec_spec.toteng(bound_type = 'eng', bound_arr = np.array([100,1e13]))), '\n')
        
#         print('ENERGIES:')
#         print(out_highengphot_specs[-1].toteng())
#         for i in np.arange(500):
#             print(i, highengphot_tf.in_eng[i], highengphot_tf[i].toteng())
    
#         print('\n SHOULD ALSO BE ONE (JUST HIGH ENERGY): ', 
#               out_highengphot_specs[-1].toteng()/(
#         next_highengphot_spec.toteng()), '\n')
        
#         output_eng = next_highengphot_spec.N*next_highengphot_spec.eng/norm_fac(rs)
#         output_N   = next_highengphot_spec.dNdE/norm_fac(rs)
#         utils.compare_arr([next_highengphot_spec.eng, output_eng])
#         print(output_eng[385:391]/1e9)
#         print(output_N[385:391])
#         print(next_highengphot_spec.eng[385:391])
        
#         utils.compare_arr([out_highengphot_specs[-1].N, next_highengphot_spec.N])
        
        # Re-define existing variables.
        prev_rs = rs
        rs = np.exp(np.log(rs) - 0.002*coarsen_factor)
        
        dt = dlnz*coarsen_factor/phys.hubble(rs)
        next_highengphot_spec.rs = rs
        next_lowengphot_spec.rs  = rs
        next_lowengelec_spec.rs  = rs
        
        # Add the next injection spectrum to next_highengphot_spec
    
        (ics_sec_phot_tf, ics_sec_elec_tf, continuum_loss) = get_ics_cooling_tf(
            ics_thomson_ref_tf_2, ics_rel_ref_tf_2, engloss_ref_tf_2,
            eleceng, photeng, rs, fast=True
        )

        ics_phot_spec = ics_sec_phot_tf.sum_specs(in_spec_elec) * norm_fac(rs)
        ics_lowengelec_spec = ics_sec_elec_tf.sum_specs(in_spec_elec) * norm_fac(rs)
        positronium_phot_spec = pos.weighted_photon_spec(photeng)*in_spec_elec.totN()/2 * norm_fac(rs)
        if positronium_phot_spec.spec_type != 'N':
            positronium_phot_spec.switch_spec_type()
    

        next_inj_spec = in_spec_phot * norm_fac(rs) + ics_phot_spec + positronium_phot_spec

        # This keeps the redshift. 
        next_highengphot_spec.N += next_inj_spec.N
        
        append_highengphot_spec(next_highengphot_spec)
        append_lowengphot_spec(next_lowengphot_spec)
        append_lowengelec_spec(next_lowengelec_spec)
        
        print('************************ END OF LOOP **************************')
        
        
    return (
        xe_arr, T_m_arr, 
        out_highengphot_specs, out_lowengphot_specs, out_lowengelec_specs
    )

In [49]:
%%prun
# a = evolve(
#         phot_spec_init*0, phot_spec_init, 
#         rate_func_decay_N, rate_func_decay_eng, .9999, phys.TCMB(phot_spec_init.rs),
#         highengphot_tf_interp, lowengphot_tf_interp, 
#         lowengelec_tf_interp, 
#         end_rs, coarsen_factor=1, std_soln=False
#     )

b_t = evolve(
        elec_spec_init, phot_spec_init*0, 
        rate_func_decay_N, rate_func_decay_eng, xe_std(phot_spec_init.rs), phys.TCMB(phot_spec_init.rs),
        #rate_func_decay_N, rate_func_decay_eng, 2e-4, phys.TCMB(phot_spec_init.rs),
        highengphot_tf_interp, lowengphot_tf_interp, 
        lowengelec_tf_interp, 
        end_rs, coarsen_factor=10, std_soln=True
    )

input electron energy:  3.7837473762910796e-05
ICS secondary photon spectrum energy:  3.994717470007986e-05
Upscattered photon energy:  4.0538621420277805e-06
ICS low energy electron spectrum energy:  9.775092040538839e-07
Positronium spectrum energy:  1.9334909215821448e-06


HBox(children=(IntProgress(value=0, max=331), HTML(value='')))

T_m at 3000: 0.70459598745
x_e at 3000: 0.999999999999
Standard x_e at 3000: 0.999999999999
Standard T_m at 3000: 0.70459598745
************************ END OF LOOP **************************
T_m at 2940.5960199202646: 0.6906438692007426
x_e at 2940.5960199202646: 0.9999999961373496
Standard x_e at 2940.5960199202646: 0.999999996042519
Standard T_m at 2940.5960199202646: 0.6906438690259646
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003097593561002737, 0.019479701178527344, 0.11317817191716949, 7.891934693259544e-16, 0.7891934673151011]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003097593561002737, 0.019479701178527344, 0.11317817191716949, 7.892014254603008e-16, 0.7891934673151011]


../darkhistory/spec/spectrum.py:748: RuntimeWarning: The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.
  warnings.warn("The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.", RuntimeWarning)
../darkhistory/spec/spectrum.py:662: RuntimeWarning: divide by zero encountered in log
  np.log(bound_arr),


************************ END OF LOOP **************************
T_m at 2882.3683174569696: 0.676968195934459
x_e at 2882.3683174569696: 0.9999999944083438
Standard x_e at 2882.3683174569696: 0.9999999942823633
Standard T_m at 2882.3683174569696: 0.6769681957447857
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003171896378754461, 0.01981254568421192, 0.11431555742771173, 7.906146966596214e-16, 0.7906069209519752]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003171896378754461, 0.01981254568421192, 0.11431555742771173, 7.906184308684415e-16, 0.7906069209519752]
************************ END OF LOOP **************************
T_m at 2825.2936007527474: 0.663563318914999
x_e at 2825.2936007527474: 0.9999999918396616
Standard x_e at 2825.2936007527474: 0.9999999916727261
Standard T_m at 2825.2936007527474: 0.6635633187094041
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0031480178484815787, 0.019746492265808845, 0.11479488541436592, 7.910941834484404e-16

...

************************ END OF LOOP **************************
T_m at 14.102718322751006: 0.02770949919663413
x_e at 14.102718322751006: 0.03627228922870207
Standard x_e at 14.102718322751006: 0.00020514231930223047
Standard T_m at 14.102718322751006: 0.00036781096506189443
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0074001740758249885, 0.042108142550504324, 0.0859511434470579, 6.927193243423937e-06, 0.011838703990680164]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.00786113341856248, 0.04310479307524421, 0.08618260894527599, 9.597255755769974e-06, 0.010146958562472342]
************************ END OF LOOP **************************
T_m at 13.823465789979405: 0.02753678118565889
x_e at 13.823465789979405: 0.036786063392177104
Standard x_e at 13.823465789979405: 0.00020475231295671127
Standard T_m at 13.823465789979405: 0.00035366368140214065
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.007256770186594022, 0.041085240553315444, 0.08311949426541

************************ END OF LOOP **************************
T_m at 10.658605218664967: 0.023929923120421397
x_e at 10.658605218664967: 0.04256416797130097
Standard x_e at 10.658605218664967: 0.00020000671503852337
Standard T_m at 10.658605218664967: 0.0002116135631809146
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.005269970949761591, 0.027618039707894875, 0.04990398756771399, 6.039051963039407e-06, 0.004562042844277831]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.005391659002347408, 0.02788291983562885, 0.04999204134776967, 8.494884955064727e-06, 0.004084965050910336]
************************ END OF LOOP **************************
T_m at 10.447550694635858: 0.023562348163822775
x_e at 10.447550694635858: 0.04291735056813628
Standard x_e at 10.447550694635858: 0.00019966330835994713
Standard T_m at 10.447550694635858: 0.00020343792861525514
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.005112365517798555, 0.02660940313427185, 0.04768018876700

************************ END OF LOOP **************************
T_m at 8.055600530862128: 0.018139166556067385
x_e at 8.055600530862128: 0.04592524166556172
Standard x_e at 8.055600530862128: 0.0001954820017486127
Standard T_m at 8.055600530862128: 0.00012119401125128187
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.00314473479841718, 0.01478204275690865, 0.02389209005421928, 5.513261142164178e-06, 0.0014499165555172254]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003183972129902685, 0.014868813002129862, 0.023951087950757132, 7.887485012734845e-06, 0.0012625368584020871]
************************ END OF LOOP **************************
T_m at 7.896088953040251: 0.01769929914375945
x_e at 7.896088953040251: 0.04602586355560173
Standard x_e at 7.896088953040251: 0.00019517532370913316
Standard T_m at 7.896088953040251: 0.00011654481788186266
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.003005966507261042, 0.01400374135011545, 0.022467324276414537, 5

************************ END OF LOOP **************************
T_m at 6.088291908887705: 0.012268613826397847
x_e at 6.088291908887705: 0.04567650786346722
Standard x_e at 6.088291908887705: 0.00019145873007359512
Standard T_m at 6.088291908887705: 6.949001130511982e-05
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0014780644573266215, 0.006043118139907676, 0.00888601362125148, 5.239439146002005e-06, 0.0004455924966805301]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0015034588622572067, 0.006099853168584786, 0.008937138804064458, 7.47253585108732e-06, 0.0003101047835547728]
************************ END OF LOOP **************************
T_m at 5.967735651795982: 0.011892202372382338
x_e at 5.967735651795982: 0.04553495267349278
Standard x_e at 5.967735651795982: 0.0001911936926448514
Standard T_m at 5.967735651795982: 6.668987658516577e-05
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0013852584633333221, 0.005599659692693228, 0.00818479292282440

************************ END OF LOOP **************************
T_m at 4.60143203797377: 0.007790634654020987
x_e at 4.60143203797377: 0.042698701646198545
Standard x_e at 4.60143203797377: 0.0001878558762009728
Standard T_m at 4.60143203797377: 4.0116461516929206e-05
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0005110741162212712, 0.0017530799814151985, 0.0023975948147791993, 5.13224958102258e-06, 0.00021174071868800505]
Standard f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0005331858701475838, 0.0018025752149593898, 0.002443367778396944, 7.185111330784989e-06, 9.230790584999452e-05]
************************ END OF LOOP **************************
T_m at 4.510317578933088: 0.007538748501435927
x_e at 4.510317578933088: 0.0424296566818394
Standard x_e at 4.510317578933088: 0.00018761453257739815
Standard T_m at 4.510317578933088: 3.8583259180249866e-05
Back Reaction f_cont, f_lyman, f_ionH, f_ionHe, f_heat:  [0.0004674932206300991, 0.0015814936409442263, 0.0021538425516

In [15]:
plt.figure()
# plt.plot(a[3].rs, a[0])
plt.plot(b[3].rs, b[0])
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


plt.figure()
# plt.plot(a[3].rs, a[1]/phys.kB)
plt.plot(b[3].rs, b[1]/phys.kB)
plt.plot(b[3].rs, phys.TCMB(b[3].rs)/phys.kB, 'k:')
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')


NameError: name 'b' is not defined

<Figure size 720x720 with 0 Axes>

In [ ]:
test_tf = highengphot_tf_interp.get_tf(4, xe_std(3000))
highengphot_tf_interp.xe

In [ ]:
plt.figure()

ax = plt.gca()

print(test_tf.in_eng[375])

test_tf.plot(ax, ind=250, indtype='ind')
test_tf.plot(ax, ind=377, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
test_spectra = highengphot_tf_interp._grid_vals[-1,-1,:,:]

plt.figure()
ax = plt.gca()

plt.plot(highengphot_tf_interp.eng, test_spectra[370, :])

ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
tf_before_interp = highengphot_tflist_arr[-1][-1]
tf_before_interp_2 = highengphot_tflist_arr[-1][0]

# print(tf_before_interp.in_eng[376])

plt.figure()
ax = plt.gca()
# tf_before_interp.plot(ax, ind=200, indtype='ind')
tf_before_interp.plot(ax, ind=370, indtype='ind')
# tf_before_interp_2.plot(ax, ind=376, indtype='ind')


ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 7e12, 1e-42, 1e7])

In [ ]:
lowengphot_spectra = b[3]
print(lowengphot_spectra.in_eng)

lowengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengphot_spectra[0].toteng())

plt.figure()
ax = plt.gca()

lowengphot_spectra.switch_spec_type()
lowengphot_spectra.plot(ax, ind=0, indtype='ind')
lowengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4, 1e3, 1e-10, 1e6])

In [ ]:
lowengelec_spectra = b[4]

lowengelec_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(lowengelec_spectra[0].toteng())
utils.compare_arr([lowengelec_spectra[0].eng, lowengelec_spectra[0].dNdE])

plt.figure()
ax = plt.gca()

lowengelec_spectra.switch_spec_type()
lowengelec_spectra.plot(ax, ind=0, indtype='ind')
lowengelec_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1, 3e3, 1e-4, 1e3])

In [ ]:
highengphot_spectra = b[2]

highengphot_spectra *= (rate_func_decay_N(3000) * (0.002/phys.hubble(3000)) / (phys.nB * 3000**3))**(-1)

print(2*10**9.104)
print(highengphot_spectra[0].toteng())
print(highengphot_spectra[1].toteng())

plt.figure()
ax = plt.gca()

highengphot_spectra.switch_spec_type()
highengphot_spectra.plot(ax, ind=1, indtype='ind')
highengphot_spectra.switch_spec_type()

ax.set_xscale('log')
ax.set_yscale('log')

plt.axis([1e-4,5e12, 1e-40, 1e10])